In [115]:
# Imports
import numpy as np
from collections import Counter

In [140]:
# Load file
data = np.genfromtxt("./scenario3/run2.txt", delimiter=",")

## CoM

In [141]:
# CoM error between prediction and real
real_com = data[:, :2]
predicted_com = data[:, 2:4]

In [142]:
com_diff = np.abs(real_com - predicted_com)

In [143]:
print("Highest error between real and predicted com: ", np.max(com_diff[:, 0]), np.max(com_diff[:, 1]))
print("Lowest error between real and predicted com: ", np.min(com_diff[:, 0]), np.min(com_diff[:, 1]))
print("Average error between real and predicted com: ", np.mean(com_diff[:, 0]), np.mean(com_diff[:, 1]))

Highest error between real and predicted com:  0.0693100000000002 0.0812489
Lowest error between real and predicted com:  0.0008650000000000047 0.0003963000000000022
Average error between real and predicted com:  0.030847935802469147 0.01943404401358025


## Footsteps

In [144]:
footsteps_diff = np.abs(data[:, 4:12] - data[:, 12:20])
new_idxs = np.delete(np.where(footsteps_diff[:, 0] >= 0.0)[0], np.where(footsteps_diff[:, 0] > 0.04)[0])
footsteps_diff = footsteps_diff[new_idxs, :]

In [145]:
real_footsteps = data[new_idxs, 4:12]
predicted_footsteps = data[new_idxs, 12:20]

In [146]:
# Swinging booleans
fr_rl_swinging = np.where(real_footsteps[:, 2] > real_footsteps[:, 0])[0]
fl_rr_swinging = np.where(real_footsteps[:, 0] > real_footsteps[:, 2])[0]
print(fr_rl_swinging.shape)
print(fl_rr_swinging.shape)

(36,)
(39,)


In [147]:
new_idxs.shape

(75,)

In [148]:
print("Highest error for FL foot: ", np.max(footsteps_diff[fl_rr_swinging, 0]), np.max(footsteps_diff[fl_rr_swinging, 1]))
print("Lowest error for FL foot: ", np.min(footsteps_diff[fl_rr_swinging, 0]), np.min(footsteps_diff[fl_rr_swinging, 1]))
print("Average error for FL foot: ", np.mean(footsteps_diff[fl_rr_swinging, 0]), np.mean(footsteps_diff[fl_rr_swinging, 1]))

Highest error for FL foot:  0.02273999999999976 0.050692000000000015
Lowest error for FL foot:  0.000140000000000029 0.00010400000000000686
Average error for FL foot:  0.006029487179487267 0.012130610256410258


In [149]:
print("Highest error for FR foot: ", np.max(footsteps_diff[fl_rr_swinging, 2]), np.max(footsteps_diff[fr_rl_swinging, 3]))
print("Lowest error for FR foot: ", np.min(footsteps_diff[fr_rl_swinging, 2]), np.min(footsteps_diff[fr_rl_swinging, 3]))
print("Average error for FR foot: ", np.mean(footsteps_diff[fr_rl_swinging, 2]), np.mean(footsteps_diff[fr_rl_swinging, 3]))

Highest error for FR foot:  0.016750000000000043 0.0743
Lowest error for FR foot:  0.0003750000000000142 6.80000000000125e-05
Average error for FR foot:  0.01056261111111116 0.012512805555555555


In [150]:
print("Highest error for RL foot: ", np.max(footsteps_diff[fr_rl_swinging, 4]), np.max(footsteps_diff[fr_rl_swinging, 5]))
print("Lowest error for RL foot: ", np.min(footsteps_diff[fr_rl_swinging, 4]), np.min(footsteps_diff[fr_rl_swinging, 5]))
print("Average error for RL foot: ", np.mean(footsteps_diff[fr_rl_swinging, 4]), np.mean(footsteps_diff[fr_rl_swinging, 5]))

Highest error for RL foot:  0.03404000000000007 0.038439299999999996
Lowest error for RL foot:  0.000580000000000247 0.0009479999999999905
Average error for RL foot:  0.009947786111111145 0.011232169444444445


In [151]:
print("Highest error for RR foot: ", np.max(footsteps_diff[fl_rr_swinging, 6]), np.max(footsteps_diff[fl_rr_swinging, 7]))
print("Lowest error for RR foot: ", np.min(footsteps_diff[fl_rr_swinging, 6]), np.min(footsteps_diff[fl_rr_swinging, 7]))
print("Average error for RR foot: ", np.mean(footsteps_diff[fl_rr_swinging, 6]), np.mean(footsteps_diff[fl_rr_swinging, 7]))

Highest error for RR foot:  0.020412000000000097 0.04541399999999998
Lowest error for RR foot:  8.00000000005241e-05 0.0002479999999999982
Average error for RR foot:  0.00620888205128207 0.012691333333333334


## Edges

In [152]:
# X coordinates of the obstaces edge
stairs = 13
depth = 3/stairs
edges_up = [1 + depth * x for x in range(stairs)] 
edges_down = [(edges_up[-1] + 2 * depth) + depth * x for x in range(stairs)]
edges = edges_up + edges_down

In [153]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [154]:
# Compute minimum distance from any obstacle
def get_min_edge_distances(footsteps_dataset):
    min_edge_distances = []
    for footstep in footsteps_dataset:
        temp = []
        for idx in range(len(footstep)):
            if idx in [0, 2, 4, 6]:
                temp.append(find_nearest(edges, footstep[idx]))
        min_edge_distances.append(temp)
        
    return np.array(min_edge_distances)

In [155]:
real_edge_distances = np.abs(real_footsteps[:, ::2] - get_min_edge_distances(real_footsteps))
predicted_edge_distances = np.abs(predicted_footsteps[:, ::2] - get_min_edge_distances(predicted_footsteps))

In [156]:
min_distance = 0.015

In [157]:
print("Lowest distance for FL foot: ", np.min(real_edge_distances[fl_rr_swinging, 0]), np.min(predicted_edge_distances[fl_rr_swinging, 0]))
print(f"Number of contacts with less than {min_distance} distance: ", len(np.where(real_edge_distances[fl_rr_swinging, 0] < min_distance)[0]))

Lowest distance for FL foot:  0.038866923076922966 0.0400915384615379
Number of contacts with less than 0.015 distance:  0


In [158]:
print("Lowest distance for FR foot: ", np.min(real_edge_distances[fr_rl_swinging, 1]), np.min(predicted_edge_distances[fr_rl_swinging, 1]))
print(f"Number of contacts with less than {min_distance} distance: ", len(np.where(real_edge_distances[fr_rl_swinging, 1] < min_distance)[0]))

Lowest distance for FR foot:  0.027540769230769868 0.043130769230770305
Number of contacts with less than 0.015 distance:  0


In [159]:
print("Lowest distance for RL foot: ", np.min(real_edge_distances[fr_rl_swinging, 2]), np.min(predicted_edge_distances[fr_rl_swinging, 2]))
print(f"Number of contacts with less than {min_distance} distance: ", len(np.where(real_edge_distances[fr_rl_swinging, 2] < min_distance)[0]))

Lowest distance for RL foot:  0.03162615384615375 0.048366153846153725
Number of contacts with less than 0.015 distance:  0


In [160]:
print("Lowest distance for RR foot: ", np.min(real_edge_distances[fl_rr_swinging, 3]), np.min(predicted_edge_distances[fl_rr_swinging, 3]))
print(f"Number of contacts with less than {min_distance} distance: ", len(np.where(real_edge_distances[fl_rr_swinging, 3] < min_distance)[0]))

Lowest distance for RR foot:  0.045667692307691965 0.051637692307691996
Number of contacts with less than 0.015 distance:  0


## Velocities

In [165]:
print("Velocities commanded: ", set(data[:, -1]))

Velocities commanded:  {0.9, 0.8, 0.3, 0.5, 0.4, 0.7}


In [166]:
print("Cound of velocities: ", Counter(data[:, -1]))

Cound of velocities:  Counter({0.9: 66, 0.4: 6, 0.3: 4, 0.5: 3, 0.8: 1, 0.7: 1})
